In [3]:
using Toolips

In [4]:
comps = components(h("Hello", 1, text = "world", align = "center"), divider("hello"), divider())

3-element Vector{Servable}:
 Component("Hello", Toolips.var"#f#1"{String, String, Dict{Any, Any}, Vector{Servable}}("Hello", "h1", Dict{Any, Any}(:children => Servable[], :text => "world", :align => "center"), Servable[]), Dict{Any, Any}(:children => Servable[], :text => "world", :align => "center"), Servable[], "h1")
 Component("hello", Toolips.var"#f#1"{String, String, Dict{Any, Any}, Vector{Servable}}("hello", "div", Dict{Any, Any}(:children => Servable[]), Servable[]), Dict{Any, Any}(:children => Servable[]), Servable[], "div")
 Component("", Toolips.var"#f#1"{String, String, Dict{Any, Any}, Vector{Servable}}("", "div", Dict{Any, Any}(:children => Servable[]), Servable[]), Dict{Any, Any}(:children => Servable[]), Servable[], "div")

In [5]:
sc = Toolips.SpoofConnection()
write!(sc, comps)

In [109]:
compdata = "<h1 id=Hello  align=center>world</h1><div id=hello ></div><div id=hello ></div><div></div>

"<h1 id=Hello  align=center>world</h1><div id=hello ></div><div id=hello ></div><div></div>"

In [155]:
function compparser(s::String)
    tagpos::Vector{UnitRange{Int64}} = [f[1]:e[1] for (f, e) in zip(findall("<", s), findall(">", s))]
    comps = Vector{Servable}()
    for tag::UnitRange in tagpos
       if contains(s[tag], "/") || ~(contains(s[tag], " id="))
            continue
        end
        tagr::UnitRange = findnext(" ", s, tag[1])
        nametag::String = s[minimum(tagr):maximum(tagr)]
        textr::UnitRange = maximum(tag) + 1:findnext("<", s, maximum(tag))[1] - 1
        tagtext::String = s[textr]
        propvec = split(s[maximum(tagr) + 1:maximum(tag) - 1], " ")
        properties = Dict()
        for segment in propvec
            ppair = split(segment, "=")
            if length(ppair) != 2
                continue
            end
            push!(properties, string(ppair[1]) => string(ppair[2]))
        end
        name::String = properties["id"]
        delete!(properties, "id")
        properties["text"] = tagtext
        push!(comps, Component(name, string(name), properties))
    end
    return(comps)::Vector{Servable}
end

compparser (generic function with 1 method)

In [156]:
compdata[27:43]

">world</h1><div i"

In [157]:
x = compparser(compdata)

3-element Vector{Servable}:
 Component("Hello", Toolips.var"#f#1"{String, String, Dict{Any, Any}, Vector{Servable}}("Hello", "Hello", Dict{Any, Any}(:children => Servable[], "text" => "world", "align" => "center"), Servable[]), Dict{Any, Any}(:children => Servable[], "text" => "world", "align" => "center"), Servable[], "Hello")
 Component("hello", Toolips.var"#f#1"{String, String, Dict{Any, Any}, Vector{Servable}}("hello", "hello", Dict{Any, Any}(:children => Servable[], "text" => ""), Servable[]), Dict{Any, Any}(:children => Servable[], "text" => ""), Servable[], "hello")
 Component("hello", Toolips.var"#f#1"{String, String, Dict{Any, Any}, Vector{Servable}}("hello", "hello", Dict{Any, Any}(:children => Servable[], "text" => ""), Servable[]), Dict{Any, Any}(:children => Servable[], "text" => ""), Servable[], "hello")

In [144]:
x[1].properties

Dict{Any, Any} with 3 entries:
  :children => Servable[]
  "text"    => "world"
  "align"   => "center"

In [129]:
x[2]

Component("hello", Toolips.var"#f#1"{String, String, Dict{Any, Any}, Vector{Servable}}("hello", "hello", Dict{Any, Any}(:children => Servable[], "text" => ""), Servable[]), Dict{Any, Any}(:children => Servable[], "text" => ""), Servable[], "hello")

In [130]:
x[3].name

"hello"